In [ ]:
#####################################################
# Author: Prof. Marc Jofre
# Dept. Network Engineering
# Universitat Politècnica de Catalunya - Technical University of Catalonia
#
# Modified: 2024
# Created: 2024
#
# Script Quantum LAN application Server layer
#####################################################
import sys, os

pathScriptBelowAgentScript='./CppScripts/'
sys.path.append(pathScriptBelowAgentScript)
pathScriptBelowAgentScript='./JupyterPythonScripts/'
sys.path.append(pathScriptBelowAgentScript)

import QpresentationLayerAgent

#####################################################
# Parameters
## Network
IPnodeServerConNet = "192.168.9.2" # IP address of the server node (connected to the server host) in the control/configuration network
IPhostClientOpNet = "10.0.0.2" # IP address of the host client (connected to the server host) in the operational network

ParamsServerListChar = [IPnodeServerConNet,IPhostClientOpNet,"server"]


In [ ]:
##############################################################
# Functions
def ListCharArrayParser(ListCharArrayAux):
    # Actually concatenating a python list of strings to a single string
    ParsedCharArrayAux=",".join(ListCharArrayAux)
    return ParsedCharArrayAux
    

In [ ]:
# The parameters passed and received from agents are structured
ParamsAscendingCharArray = "none"
ParamsDescendingCharArray = ListCharArrayParser(ParamsServerListChar)

In [ ]:
# Create instance of Agent below
QPLAagent = QpresentationLayerAgent.QPLA(ParamsDescendingCharArray,ParamsAscendingCharArray)

In [ ]:
### RUN UP TO HERE (INCLUDED) TO INITIALIZE CONNECTIONS AND THREAD QtransportLayerAgentH thread #####
# At this point, in this order, the node in the server has been initialized, the node in the client, the server host (up to this equivalent point) and then the client host (up to here)
### Initiate thread process in QtransportLayerAgentH
QPLAagent.InitAgentProcess() # Initialize the thread in QtransportLayerAgentH

In [ ]:
#####################
# Some tests to send information to the different elements
messagePayloadAux="Host server sends information to attached node"
messageAuxChar = ListCharArrayParser([IPnodeServerConNet,messagePayloadAux])
QPLAagent.SendMessageAgent(messageAuxChar)
messagePayloadAux="Host server sends information to client host"
messageAuxChar = ListCharArrayParser([IPhostClientOpNet,messagePayloadAux])
QPLAagent.SendMessageAgent(messageAuxChar)
#messagePayloadAux="Host server sends information to client host attached node"
#messageAuxChar = ListCharArrayParser([IPnodeServerConNet,messagePayloadAux])
#QPLAagent.SendMessageAgent(messageAuxChar)
#messagePayloadAux="Node host sends information to node client"
#messageAuxChar = ListCharArrayParser([IPnodeServerConNet,messagePayloadAux])
#QPLAagent.SendMessageAgent(messageAuxChar)
#messagePayloadAux="Host server sends information to host server process"
#messageAuxChar = ListCharArrayParser([IPnodeServerConNet,messagePayloadAux])
#QPLAagent.SendMessageAgent(messageAuxCha